In [1]:
import numba

import datetime
import pandas as pd
import numpy as np
#import pandas_profiling
import sys
import os

import time
from datetime import date
from datetime import timedelta
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import neighbors
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor
from scipy import stats
import sklearn.metrics
from random import randrange
from itertools import chain
import random

from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(font_scale=2)

pd.set_option('display.max_columns', 600)
pd.set_option('display.max_rows', 600)

In [2]:
df = pd.read_csv('HORSE_DATA_ORIG.csv')

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (546) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Dummy columns for Result (Winner/Loser/Removed)

In [3]:
df = pd.concat([df,pd.get_dummies(df.HORSE_RESULT)], axis = 1)

## Normalised Price and Amnt backed.
###      Grouped by race, Feature is normalised to the lowest price and amount backed.

In [4]:
market_list = df.groupby('MARKETID').count().index.tolist()

feat_list = ['AMT_BACKED_1_MIN_OUT',\
             'AMT_BACKED_10_MINS_OUT',\
             'AMT_BACKED_20_MINS_OUT',\
             'AMT_BACKED_2_SECS_OUT',\
             'AMT_BACKED_30_MINS_OUT',\
             'AMT_BACKED_40_MINS_OUT',\
             'AMT_BACKED_45_MINS_OUT',\
             'AMT_BACKED_5_MINS_OUT',\
             'AMT_BACKED_60_MINS_OUT',\
             'PRICE_10_MINS_OUT',\
             'PRICE_1_MIN_OUT',\
             'PRICE_20_MINS_OUT',\
             'PRICE_2_SECS_OUT',\
             'PRICE_30_MINS_OUT',\
             'PRICE_40_MINS_OUT',\
             'PRICE_45_MINS_OUT',\
             'PRICE_5_MINS_OUT',\
             'PRICE_60_MINS_OUT',\
             'BACKPRICE']

for feat in feat_list:

    met_list = []
    for m_ID in market_list:

        fav_price = df[df.MARKETID == m_ID][feat].min()
        met_list.append((df[df.MARKETID == m_ID][feat]/fav_price).tolist())

    met_list = list(chain.from_iterable(met_list))

    df[feat + '_NORM'] = met_list

## Horse Form split into separate columns.
### Race 1 is the most recent race.

### link explainging the letter meanings in horse form.
https://help.racingpost.com/hc/en-us/articles/115001699689-Abbreviations-on-the-racecard

In [9]:
df.HORSE_FORM = df.HORSE_FORM.fillna('0')

def form_split(form):
    s = ''

    for i in range(6):
        try:
            s = s + list(form)[i] + ' '
        except:
            s = s + '0 '
    return s

df['HORSE_FORM'] = df.apply(lambda row: form_split(row['HORSE_FORM']), axis=1)


df[['Race_6','Race_5','Race_4','Race_3','Race_2','Race_1']] = df.HORSE_FORM.str.split(" ",n = 5, expand=True,)

## Trainer/Jockey/Owner Ranking/Score Elo points....

In [10]:
df.to_csv('HORSE_DATA_MOD.csv')